In [2]:
import pandas as pd

In [3]:
evictions=pd.read_csv('/Users/renasong/Documents/2020su/proj/Eviction_Notices.csv')

/Users/renasong/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
evictions=evictions[['Address','Eviction Notice Source Zipcode','File Date','Neighborhoods - Analysis Boundaries','Location','Non Payment',
                    'Breach','Nuisance','Illegal Use','Failure to Sign Renewal','Access Denial','Unapproved Subtenant',
                     'Owner Move In','Demolition','Capital Improvement','Substantial Rehab','Ellis Act WithDrawal',
                     'Condo Conversion','Roommate Same Unit','Other Cause','Late Payments','Lead Remediation','Development',
                     'Good Samaritan Ends']]

evictions=evictions.rename(columns={'Neighborhoods - Analysis Boundaries':'Neighborhood','Eviction Notice Source Zipcode':'Zipcode'})
